# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import json

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [2]:
url = 'https://scanner.tradingview.com/america/scan'
payload = {"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":True},{"left":"active_symbol","operation":"equal","right":True}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","Perf.W","Perf.1M","Perf.3M","Perf.6M","Perf.Y","average_volume_90d_calc","EMA30","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","currency","fundamental_currency_code"],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"},"range":[0,8049]}
data = requests.post(url, json = payload)

In [3]:
stocks_data = data.json()['data']
len(stocks_data)

7052

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
stocks_data[0]

{'s': 'NASDAQ:AAPL',
 'd': ['apple',
  'AAPL',
  152.78,
  4.05230539,
  0.13436015,
  7.31947176,
  -1.73023734,
  -3.80304748,
  76227232.94444445,
  148.13504817,
  'Apple Inc.',
  'stock',
  'common',
  'delayed_streaming_900',
  100,
  1,
  'false',
  0,
  'USD',
  'USD']}

In [5]:
stats_from_tradingview = []

for data in stocks_data:
    stock = {}
    stock['Symbol'] = data['d'][1]
    stock['Name'] = data['d'][10]
    stock['Price'] = data['d'][2]
    stock['weeklyPerformance'] = data['d'][3]
    stock['1monthPerformace'] = data['d'][4]
    stock['3monthPerformace'] = data['d'][5]
    stock['6monthPerformace'] = data['d'][6]
    stock['1yearPerformace'] = data['d'][7]
    stock['AvgVolume'] = data['d'][8]
    stock['Exp Moving Average'] = data['d'][9]
    
    stats_from_tradingview.append(stock)

In [6]:
stats_from_tradingview[0]

{'Symbol': 'AAPL',
 'Name': 'Apple Inc.',
 'Price': 152.78,
 'weeklyPerformance': 4.05230539,
 '1monthPerformace': 0.13436015,
 '3monthPerformace': 7.31947176,
 '6monthPerformace': -1.73023734,
 '1yearPerformace': -3.80304748,
 'AvgVolume': 76227232.94444445,
 'Exp Moving Average': 148.13504817}

In [7]:
# Serializing json
json_object = json.dumps(stats_from_tradingview)
 
# Writing to sample.json
with open("stocks_all_performance.json", "w") as outfile:
    outfile.write(json_object)

In [8]:
hqm_dataframe = pd.read_json('stocks_all_performance.json')
available_stocks = pd.read_json('curated_stocks_all.json')
available_stocks_list = available_stocks['symbol'].tolist()

In [9]:
hqm_dataframe = hqm_dataframe[hqm_dataframe['Symbol'].isin(available_stocks_list)]

In [10]:
mean_val = hqm_dataframe['AvgVolume'].mean()
hqm_dataframe['AvgVolume'].fillna(value=mean_val, inplace=True)
hqm_dataframe[hqm_dataframe['1yearPerformace'].isna()]

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average


In [11]:
hqm_dataframe['Exp Moving Average'].fillna(hqm_dataframe['Price']+1, inplace=True)


In [12]:
hqm_dataframe['Diff From Moving Avg'] = hqm_dataframe['Exp Moving Average'] - hqm_dataframe['Price']
hqm_dataframe['Diff From Moving Avg'] = hqm_dataframe['Diff From Moving Avg'].apply(abs)

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [13]:
# for row in hqm_dataframe.index:
#     if(hqm_dataframe.loc[row, '1yearPerformace'] == None):
#         hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

In [14]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg
0,AAPL,Apple Inc.,152.7800,4.052305,0.134360,7.319472,-1.730237,-3.803047,7.622723e+07,148.135048,4.644952
1,MSFT,Microsoft Corporation,252.1200,0.542351,-2.066501,2.973370,-3.216891,-9.244060,3.035574e+07,253.044926,0.924926
2,GOOG,Alphabet Inc.,95.5224,5.947649,-6.975313,-0.175149,-13.200909,-24.338993,2.934221e+07,94.579588,0.942812
3,AMZN,"Amazon.com, Inc.",93.1600,-0.756365,-9.487491,4.392649,-28.836605,-31.842541,8.003210e+07,95.720111,2.560111
4,NVDA,NVIDIA Corporation,239.4250,3.236030,15.074978,47.130216,69.121283,12.358628,4.819118e+07,217.198151,22.226849
...,...,...,...,...,...,...,...,...,...,...,...
3073,RSLS,"ReShape Lifesciences, Inc.",3.8204,-10.108235,-41.224615,-45.617082,-80.607107,-91.784086,5.641589e+05,5.341000,1.520600
3074,CYTO,Altamira Therapeutics Ltd.,1.4492,-40.848980,-50.539249,-72.656604,-79.759777,-92.037363,3.477521e+05,2.773291,1.324091
3075,REVB,"Revelation Biosciences, Inc.",2.0800,-5.022831,-75.898030,-70.404098,-85.505226,-93.322632,1.113027e+05,3.988315,1.908315
3076,SNES,"SenesTech, Inc.",1.8700,-11.792453,-57.787810,-25.793651,-79.231453,-87.012085,1.571227e+05,2.525431,0.655431


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [15]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg
0,AAPL,Apple Inc.,152.7800,4.052305,0.134360,7.319472,-1.730237,-3.803047,7.622723e+07,148.135048,4.644952
1,MSFT,Microsoft Corporation,252.1200,0.542351,-2.066501,2.973370,-3.216891,-9.244060,3.035574e+07,253.044926,0.924926
2,GOOG,Alphabet Inc.,95.5224,5.947649,-6.975313,-0.175149,-13.200909,-24.338993,2.934221e+07,94.579588,0.942812
3,AMZN,"Amazon.com, Inc.",93.1600,-0.756365,-9.487491,4.392649,-28.836605,-31.842541,8.003210e+07,95.720111,2.560111
4,NVDA,NVIDIA Corporation,239.4250,3.236030,15.074978,47.130216,69.121283,12.358628,4.819118e+07,217.198151,22.226849
...,...,...,...,...,...,...,...,...,...,...,...
3073,RSLS,"ReShape Lifesciences, Inc.",3.8204,-10.108235,-41.224615,-45.617082,-80.607107,-91.784086,5.641589e+05,5.341000,1.520600
3074,CYTO,Altamira Therapeutics Ltd.,1.4492,-40.848980,-50.539249,-72.656604,-79.759777,-92.037363,3.477521e+05,2.773291,1.324091
3075,REVB,"Revelation Biosciences, Inc.",2.0800,-5.022831,-75.898030,-70.404098,-85.505226,-93.322632,1.113027e+05,3.988315,1.908315
3076,SNES,"SenesTech, Inc.",1.8700,-11.792453,-57.787810,-25.793651,-79.231453,-87.012085,1.571227e+05,2.525431,0.655431


In [16]:
time_periods = [
    'weeklyPerformance',
    '1monthPerformace',
    '3monthPerformace',
    '6monthPerformace',
    '1yearPerformace',
    'Diff From Moving Avg'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}'
        percentile_col = f'{time_period} Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile,Diff From Moving Avg Percentile
0,AAPL,Apple Inc.,152.7800,4.052305,0.134360,7.319472,-1.730237,-3.803047,7.622723e+07,148.135048,4.644952,85.640026,73.489279,62.410656,49.350227,56.920078,90.318389
1,MSFT,Microsoft Corporation,252.1200,0.542351,-2.066501,2.973370,-3.216891,-9.244060,3.035574e+07,253.044926,0.924926,63.255361,63.807667,52.274204,47.141001,50.519818,56.692658
2,GOOG,Alphabet Inc.,95.5224,5.947649,-6.975313,-0.175149,-13.200909,-24.338993,2.934221e+07,94.579588,0.942812,91.000650,43.762183,44.379467,33.040936,34.600390,57.504873
3,AMZN,"Amazon.com, Inc.",93.1600,-0.756365,-9.487491,4.392649,-28.836605,-31.842541,8.003210e+07,95.720111,2.560111,49.577648,34.567901,55.815465,19.785575,29.174789,80.766732
4,NVDA,NVIDIA Corporation,239.4250,3.236030,15.074978,47.130216,69.121283,12.358628,4.819118e+07,217.198151,22.226849,82.391163,95.321637,94.314490,97.141001,76.218324,99.220273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,RSLS,"ReShape Lifesciences, Inc.",3.8204,-10.108235,-41.224615,-45.617082,-80.607107,-91.784086,5.641589e+05,5.341000,1.520600,7.342430,2.631579,2.534113,1.397011,1.754386,68.648473
3074,CYTO,Altamira Therapeutics Ltd.,1.4492,-40.848980,-50.539249,-72.656604,-79.759777,-92.037363,3.477521e+05,2.773291,1.324091,0.097466,1.397011,0.487329,1.526966,1.721897,65.659519
3075,REVB,"Revelation Biosciences, Inc.",2.0800,-5.022831,-75.898030,-70.404098,-85.505226,-93.322632,1.113027e+05,3.988315,1.908315,18.356075,0.097466,0.682261,0.779727,1.559454,74.333983
3076,SNES,"SenesTech, Inc.",1.8700,-11.792453,-57.787810,-25.793651,-79.231453,-87.012085,1.571227e+05,2.525431,0.655431,5.912930,0.714750,7.927225,1.689409,2.956465,48.115660


In [17]:
hqm_dataframe['Diff From Moving Avg Percentile'] = abs(hqm_dataframe['Diff From Moving Avg Percentile'] - 100)

In [18]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile,Diff From Moving Avg Percentile
0,AAPL,Apple Inc.,152.7800,4.052305,0.134360,7.319472,-1.730237,-3.803047,7.622723e+07,148.135048,4.644952,85.640026,73.489279,62.410656,49.350227,56.920078,9.681611
1,MSFT,Microsoft Corporation,252.1200,0.542351,-2.066501,2.973370,-3.216891,-9.244060,3.035574e+07,253.044926,0.924926,63.255361,63.807667,52.274204,47.141001,50.519818,43.307342
2,GOOG,Alphabet Inc.,95.5224,5.947649,-6.975313,-0.175149,-13.200909,-24.338993,2.934221e+07,94.579588,0.942812,91.000650,43.762183,44.379467,33.040936,34.600390,42.495127
3,AMZN,"Amazon.com, Inc.",93.1600,-0.756365,-9.487491,4.392649,-28.836605,-31.842541,8.003210e+07,95.720111,2.560111,49.577648,34.567901,55.815465,19.785575,29.174789,19.233268
4,NVDA,NVIDIA Corporation,239.4250,3.236030,15.074978,47.130216,69.121283,12.358628,4.819118e+07,217.198151,22.226849,82.391163,95.321637,94.314490,97.141001,76.218324,0.779727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,RSLS,"ReShape Lifesciences, Inc.",3.8204,-10.108235,-41.224615,-45.617082,-80.607107,-91.784086,5.641589e+05,5.341000,1.520600,7.342430,2.631579,2.534113,1.397011,1.754386,31.351527
3074,CYTO,Altamira Therapeutics Ltd.,1.4492,-40.848980,-50.539249,-72.656604,-79.759777,-92.037363,3.477521e+05,2.773291,1.324091,0.097466,1.397011,0.487329,1.526966,1.721897,34.340481
3075,REVB,"Revelation Biosciences, Inc.",2.0800,-5.022831,-75.898030,-70.404098,-85.505226,-93.322632,1.113027e+05,3.988315,1.908315,18.356075,0.097466,0.682261,0.779727,1.559454,25.666017
3076,SNES,"SenesTech, Inc.",1.8700,-11.792453,-57.787810,-25.793651,-79.231453,-87.012085,1.571227e+05,2.525431,0.655431,5.912930,0.714750,7.927225,1.689409,2.956465,51.884340


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [19]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)   

In [20]:
hqm_dataframe

,Symbol,Name,Price,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,AvgVolume,Exp Moving Average,Diff From Moving Avg,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile,Diff From Moving Avg Percentile,HQM Score
0,AAPL,Apple Inc.,152.7800,4.052305,0.134360,7.319472,-1.730237,-3.803047,7.622723e+07,148.135048,4.644952,85.640026,73.489279,62.410656,49.350227,56.920078,9.681611,56.248646
1,MSFT,Microsoft Corporation,252.1200,0.542351,-2.066501,2.973370,-3.216891,-9.244060,3.035574e+07,253.044926,0.924926,63.255361,63.807667,52.274204,47.141001,50.519818,43.307342,53.384232
2,GOOG,Alphabet Inc.,95.5224,5.947649,-6.975313,-0.175149,-13.200909,-24.338993,2.934221e+07,94.579588,0.942812,91.000650,43.762183,44.379467,33.040936,34.600390,42.495127,48.213125
3,AMZN,"Amazon.com, Inc.",93.1600,-0.756365,-9.487491,4.392649,-28.836605,-31.842541,8.003210e+07,95.720111,2.560111,49.577648,34.567901,55.815465,19.785575,29.174789,19.233268,34.692441
4,NVDA,NVIDIA Corporation,239.4250,3.236030,15.074978,47.130216,69.121283,12.358628,4.819118e+07,217.198151,22.226849,82.391163,95.321637,94.314490,97.141001,76.218324,0.779727,74.361057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,RSLS,"ReShape Lifesciences, Inc.",3.8204,-10.108235,-41.224615,-45.617082,-80.607107,-91.784086,5.641589e+05,5.341000,1.520600,7.342430,2.631579,2.534113,1.397011,1.754386,31.351527,7.835174
3074,CYTO,Altamira Therapeutics Ltd.,1.4492,-40.848980,-50.539249,-72.656604,-79.759777,-92.037363,3.477521e+05,2.773291,1.324091,0.097466,1.397011,0.487329,1.526966,1.721897,34.340481,6.595192
3075,REVB,"Revelation Biosciences, Inc.",2.0800,-5.022831,-75.898030,-70.404098,-85.505226,-93.322632,1.113027e+05,3.988315,1.908315,18.356075,0.097466,0.682261,0.779727,1.559454,25.666017,7.856833
3076,SNES,"SenesTech, Inc.",1.8700,-11.792453,-57.787810,-25.793651,-79.231453,-87.012085,1.571227e+05,2.525431,0.655431,5.912930,0.714750,7.927225,1.689409,2.956465,51.884340,11.847520


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [21]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
# hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
# hqm_dataframe[hqm_dataframe['Price'] < 10][:50]

In [22]:
# rearrange columns
cols = list(hqm_dataframe.columns.values)
hqm_dataframe = hqm_dataframe[['Symbol',
    'Name',
    'Price',
    'HQM Score',
    'AvgVolume',
    'Exp Moving Average',
    'weeklyPerformance',
    '1monthPerformace',
    '3monthPerformace',
    '6monthPerformace',
    '1yearPerformace',
    'weeklyPerformance Percentile',
    '1monthPerformace Percentile',
    '3monthPerformace Percentile',
    '6monthPerformace Percentile',
    '1yearPerformace Percentile'
]]

In [38]:
high_volume_df = hqm_dataframe[hqm_dataframe['AvgVolume'] > 1000000]

In [39]:
high_volume_df.reset_index(inplace=True, drop=True)

In [44]:
high_volume_df[(high_volume_df['HQM Score'] > 50) & (high_volume_df['HQM Score'] < 60) & (high_volume_df['Price'] < 15)]

,Symbol,Name,Price,HQM Score,AvgVolume,Exp Moving Average,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile
326,CLXT,"Calyxt, Inc.",0.370100,59.892788,1.465217e+06,0.367851,-11.880952,-5.850929,134.835025,58.704974,-64.752381,5.750487,48.148148,99.350227,95.971410,10.591293
328,IBIO,"iBio, Inc.",2.010000,59.773663,2.333867e+06,1.268183,54.615385,142.168675,195.588235,-70.763636,-73.665247,99.870045,99.935023,99.642625,3.248863,7.244964
334,CNK,"Cinemark Holdings, Inc.",13.630000,59.508339,3.268074e+06,12.599999,-0.365497,9.919355,12.737800,-1.588448,-9.854497,53.931124,91.812865,71.767381,49.675114,49.707602
340,FNB,F.N.B. Corporation,13.750000,59.102231,2.375067e+06,14.057403,-3.032440,-4.844291,4.245641,14.202658,10.887097,29.012346,52.858999,55.523067,74.756335,74.756335
343,WBD,"Warner Bros. Discovery, Inc. - Series A",14.980000,58.993935,2.608008e+07,14.706463,-3.912765,-1.447368,38.961039,18.232044,-41.689373,23.976608,66.601689,92.072775,78.914880,22.742040
347,SNAP,Snap Inc.,11.082900,58.907299,3.399146e+07,10.545097,9.191133,1.584785,16.785037,-10.980723,-63.069310,95.451592,78.200130,76.998051,35.705003,11.435997
349,ZYME,Zymeworks Inc.,8.000000,58.769222,1.052664e+06,8.525637,-2.439024,-17.012448,6.951872,60.000000,26.984127,33.901884,18.128655,61.793372,96.198830,86.354776
351,SOFI,"SoFi Technologies, Inc.",6.519000,58.679879,4.193120e+07,6.458469,-0.927052,-11.306122,49.862069,4.638844,-31.451104,48.115660,29.499675,94.671865,60.591293,29.369721
355,SWN,Southwestern Energy Company,5.420000,58.604072,2.447696e+07,5.415030,2.264151,2.264151,-11.726384,-27.540107,-5.244755,76.900585,80.116959,19.785575,20.597791,55.263158
361,ADPT,Adaptive Biotechnologies Corporation,8.792500,58.425384,1.013532e+06,8.829813,2.596266,-9.355670,9.223602,-5.151025,-25.676247,79.012346,34.990253,65.529565,44.087070,33.625731


In [26]:
top_15 = high_volume_df[:15]

### Getting Historical data

In [27]:
from yahoo_fin.stock_info import get_data
from datetime import date, timedelta
import time

In [28]:
top_15_symbols = list(top_15['Symbol'].values)
top_15_history_data = {}

end_date_today = date.today().strftime("%d/%m/%Y")
start_date_90days_before = (date.today() - timedelta(days=40)).strftime("%m/%d/%Y")

for symbol in top_15_symbols:
    top_15_history_data[symbol] = get_data(symbol, start_date = start_date_90days_before, end_date = end_date_today, interval = '1d')
    print(symbol)
    time.sleep(1)

GOTU
IQ
ASRT
CBAY
HLX
RIG
NAT
YEXT
ASXC
CHX
TK
DHT
BBIO
TGTX
SFT


In [29]:
# top_15_history_data['TAL']

In [30]:
for symbol in top_15_history_data.keys():
    top_15_history_data[symbol]['open to low diff'] = top_15_history_data[symbol]['open'] - top_15_history_data[symbol]['low']
    top_15_history_data[symbol]['open to high diff'] = top_15_history_data[symbol]['high'] - top_15_history_data[symbol]['open']
    top_15_history_data[symbol]['low to high diff'] = top_15_history_data[symbol]['high'] - top_15_history_data[symbol]['low']
    

In [31]:
# top_15_history_data['GOTU']['open to low diff'].mean()

In [32]:
for row in range(len(top_15)):
    print(top_15_history_data[top_15.loc[row, 'Symbol']]['open to low diff'].mean())
    top_15.loc[row, 'Avg open to low diff'] = top_15_history_data[top_15.loc[row, 'Symbol']]['open to low diff'].mean()
    top_15.loc[row, 'Avg open to high diff'] = top_15_history_data[top_15.loc[row, 'Symbol']]['open to high diff'].mean()
    top_15.loc[row, 'Avg low to high diff'] = top_15_history_data[top_15.loc[row, 'Symbol']]['low to high diff'].mean()
    

0.16374998433249338
0.21046432426997594
0.13316072736467635
0.1961071150643485
0.1648214033671788
0.1489285741533552
0.06392856155123029
0.11299644197736468
0.04622143081256321
0.6082143102373395
0.08428573608398438
0.11464292662484306
0.46639275550842285
0.4876784937722342
0.027857141303164617


/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [33]:
# top_15

In [34]:
# top_15_history_data['TAL']['open to high diff'].mean()

In [35]:
list(top_15.columns.values)

['Symbol',
 'Name',
 'Price',
 'HQM Score',
 'AvgVolume',
 'Exp Moving Average',
 'weeklyPerformance',
 '1monthPerformace',
 '3monthPerformace',
 '6monthPerformace',
 '1yearPerformace',
 'weeklyPerformance Percentile',
 '1monthPerformace Percentile',
 '3monthPerformace Percentile',
 '6monthPerformace Percentile',
 '1yearPerformace Percentile',
 'Avg open to low diff',
 'Avg open to high diff',
 'Avg low to high diff']

In [36]:
top_15 = top_15[[
    'Symbol',
    'Name',
    'Price',
    'HQM Score',
    'AvgVolume',
    'Exp Moving Average',
    'Avg open to low diff',
    'Avg open to high diff',
    'Avg low to high diff',
    'weeklyPerformance',
    '1monthPerformace',
    '3monthPerformace',
    '6monthPerformace',
    '1yearPerformace',
    'weeklyPerformance Percentile',
    '1monthPerformace Percentile',
    '3monthPerformace Percentile',
    '6monthPerformace Percentile',
    '1yearPerformace Percentile',
]]

In [37]:
top_15

,Symbol,Name,Price,HQM Score,AvgVolume,Exp Moving Average,Avg open to low diff,Avg open to high diff,Avg low to high diff,weeklyPerformance,1monthPerformace,3monthPerformace,6monthPerformace,1yearPerformace,weeklyPerformance Percentile,1monthPerformace Percentile,3monthPerformace Percentile,6monthPerformace Percentile,1yearPerformace Percentile
0,GOTU,Gaotu Techedu Inc.,4.8350,90.101798,6.263637e+06,3.898164,0.163750,0.234882,0.398632,38.538682,35.434174,218.092105,200.310559,212.944984,99.642625,98.797921,99.740091,99.902534,99.805068
1,IQ,"iQIYI, Inc.",7.5259,89.614468,1.931570e+07,7.042963,0.210464,0.331750,0.542214,8.286331,7.820917,126.820374,121.350000,77.497642,94.411956,89.798571,99.122807,99.122807,96.783626
2,ASRT,"Assertio Holdings, Inc.",6.1050,88.661468,1.337064e+06,5.127590,0.133161,0.172857,0.306018,8.436945,50.369458,74.928367,125.276753,175.000000,94.541910,99.707602,97.563353,99.187784,99.415205
3,CBAY,CymaBay Therapeutics Inc.,9.1100,87.069526,1.115893e+06,7.999070,0.196107,0.215714,0.411821,12.887237,6.549708,130.050505,137.239583,189.206349,97.335932,88.401559,99.252762,99.545159,99.707602
4,HLX,"Helix Energy Solutions Group, Inc.",8.4250,86.446827,1.865542e+06,8.088511,0.164821,0.195000,0.359821,2.245146,9.132124,36.326861,95.930233,66.501976,76.770630,90.968161,91.000650,98.440546,95.581546
5,RIG,Transocean Ltd (Switzerland),7.2550,85.948668,2.275930e+07,6.846569,0.148929,0.247500,0.396429,2.907801,4.088953,86.984536,96.081081,41.147860,81.026641,84.275504,98.083171,98.538012,91.877843
6,NAT,Nordic American Tankers Limited,4.6100,85.769981,3.838260e+06,3.835151,0.063929,0.118571,0.182500,2.901786,48.709677,45.425868,50.653595,97.854077,80.929175,99.610136,93.989604,94.574399,98.083171
7,YEXT,"Yext, Inc.",9.1900,85.206844,1.008976e+06,7.521072,0.112996,0.289893,0.402889,23.854447,18.123393,43.146417,101.094092,54.974705,99.025341,96.588694,93.307342,98.732943,94.476933
8,ASXC,"Asensus Surgical, Inc.",0.6929,84.004765,1.459274e+06,0.724082,0.046221,0.048350,0.094571,0.420290,15.483333,76.760204,23.732143,6.600000,62.248213,95.549058,97.693307,84.178038,69.915530
9,CHX,ChampionX Corporation,31.3000,83.912714,1.670188e+06,31.269616,0.608214,0.730714,1.338929,2.020860,7.302023,9.325882,42.014519,27.132413,75.146199,89.116309,65.659519,92.527615,86.419753
